In [58]:
from trueskill import Rating, rate
from dota import *
from config import *
import dota2api
api = dota2api.Initialise(API_KEY)

In [59]:
leagues = load_json('data/leagues.json')
# USE a league as sample data
league = next(l for l in leagues if l['leagueid'] == 1640)
league

{'name': 'joinDOTA League Season 3',
 'leagueid': 1640,
 'description': 'The global Dota 2 league for everyone. Featured are all of the matches from division 1 and 2. There are three leagues: Europe, America, and Asia. The top 10 from each league meet in the premier division. $1.00 from each ticket purchased will go directly into the prizepool.',
 'tournament_url': 'http://www.joindota.com/en/leagues/',
 'itemdef': 10742}

In [64]:
matches = load_json('data/leagues/' + str(league['leagueid']) + '.json')
teams = {}
players = {}
for match_id, match in matches.items():
    vs = []
    for i, pos in enumerate(['radiant', 'dire']):
        team_key = pos + '_team_id'
        if match[team_key] not in teams:
            team_info = api.get_team_info_by_team_id(start_at_team_id=match[team_key], teams_requested=1)['teams'][0]
            teams[match[team_key]] = {'players': {}, 'name': team_info['name']}
        team_players = []
        for player in match['players'][i * 5:5 + i * 5:1]:
            if player['account_id'] not in players:
                players[player['account_id']] = Rating()
            teams[match[team_key]]['players'][player['account_id']] = players[player['account_id']]
            team_players.append(players[player['account_id']])
        vs.append(team_players)
    result = [1, 0] if match['detail']['radiant_win'] else [0, 1]
    new_ratings = rate(vs, ranks=result)
    new_ratings = list(new_ratings[0]) + list(new_ratings[1])
    for i, rating in enumerate(new_ratings):
        players[match['players'][i]['account_id']] = rating

In [72]:
import itertools
import math
from trueskill import BETA, global_env


def win_probability(team1, team2):
    delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
    sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
    size = len(team1) + len(team2)
    denom = math.sqrt(size * (BETA * BETA) + sum_sigma)
    ts = global_env()
    return ts.cdf(delta_mu / denom)

teams.keys()

dict_keys([254140, 248456, 1395712, 1824413, 1633432, 720700, 1705097, 67, 700779, 1718551, 1321909, 992400, 795543, 55, 1161668, 1287653, 1653234, 3515, 463048, 1626847, 1220593, 1276821, 1810136, 813776, 1318686, 1276923, 1476172, 1762814, 1299826, 559133, 1839396, 1175580, 9896, 1078103, 1163855, 1819704, 1248512, 24, 1277104, 1464407, 97669, 1196773, 1595874, 1079078, 1271771, 1737926, 1080746, 1085888, 1277097, 543897, 883545, 460316, 875854, 94968, 1416475, 756782, 1266451, 576019, 1532777, 981293, 999689, 862976, 1616631, 1087276, 1317203, 1478921, 1256674, 136963, 867475, 1563135, 1055544, 1852165, 1639565, 32150, 1855667, 799536, 687124, 41995, 916115, 1255919, 314627, 1748111, 1294226, 335074, 1387750, 780202, 1282882, 1268700, 946733, 1833610, 1848225, 97215, 215271, 370469, 1291626, 41, 1589592, 888984, 177761, 1242318, 529501, 1633355, 1549365, 1199060, 1249631, 874932, 1702618, 0, 1380025, 1773197, 15321, 1709726, 1257891])

In [78]:
win_probability(list(teams[799536]['players'].values())[:5], list(teams[916115]['players'].values())[:5])

0.48061277215577886